In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [2]:
transform_test = transforms.Compose([
    transforms.ToTensor()
])

transform_train = transforms.Compose([
    transforms.ToTensor()
    ])


train_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform_train

)

test_ds = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transform_test
)


torch.manual_seed(42)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
def plot_data(dataset_augm):
    figure = plt.figure(figsize=(8, 8))
    cols, rows = 3, 3
    for i in range(1, cols * rows + 1):
        sample_idx = torch.randint(len(dataset_augm), size=(1,)).item()
        img = dataset_augm[sample_idx]
        figure.add_subplot(rows, cols, i)
        plt.axis("off")
        img = img.cpu()
        img = img.numpy()
        img = np.transpose(img, (1, 2, 0))
        plt.imshow(img.squeeze(), cmap="gray")
    plt.show()

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def train(dataloader, model, loss_fn, optimizer, beta, cutmix_prob):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # generate mixed sample
        r = np.random.rand(1)
        if beta > 0 and r < cutmix_prob:
            lam = np.random.beta(beta, beta)
            rand_index = torch.randperm(X.size()[0]).cuda()
            target_a = y
            target_b = y[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(X.size(), lam)
            X[:, :, bbx1:bbx2, bby1:bby2] = X[rand_index, :, bbx1:bbx2, bby1:bby2]
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (X.size()[-1] * X.size()[-2]))
            
            pred = model(X)
            loss = loss_fn(pred, target_a) * lam + loss_fn(pred, target_b) * (1. - lam)

        else:
            # compute output
            pred = model(X)
            loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

Using cuda device


# Net_dropout

In [9]:
lr = 1e-3
batch_size = 8
epochs = 60
beta = 1
cutmix_prob = 0.6


data = np.array([[0]*4]*epochs)
bnet_SGD_augm_history = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])

train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    test_ds, 
    batch_size=batch_size,
    shuffle=False
    )

net = Net_dropout(dropout_rate=0.1).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=lr)

for t in range(epochs):

    print(f"Epoch {t+1}\n-------------------------------")
    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer, beta, cutmix_prob)
    test_acc, test_loss = test(test_dataloader, net, criterion)

    bnet_SGD_augm_history.loc[t,"train_acc"] = train_acc        
    bnet_SGD_augm_history.loc[t,"test_acc"] = test_acc
    bnet_SGD_augm_history.loc[t,"train_loss"] = train_loss
    bnet_SGD_augm_history.loc[t,"test_loss"] = test_loss

    torch.save({
        'epoch': t+1,
        'model_state_dict': net.state_dict(),
        'train_loss': train_loss,
        'optimizer_state_dict': optimizer.state_dict()
        }, r"../model_epoch_"+str(t+1)+".pt")
        
bnet_SGD_augm_history.to_csv(r"..\bnet_augm_history.csv")
        

Epoch 1
-------------------------------
 Train Accuracy: 30.9%, Train Avg loss 1.952114 

Test Accuracy: 44.4%, Test Avg loss: 1.594613 

Epoch 2
-------------------------------
 Train Accuracy: 40.5%, Train Avg loss 1.720338 

Test Accuracy: 41.5%, Test Avg loss: 1.578735 

Epoch 3
-------------------------------
 Train Accuracy: 44.6%, Train Avg loss 1.628414 

Test Accuracy: 55.9%, Test Avg loss: 1.328350 

Epoch 4
-------------------------------
 Train Accuracy: 46.6%, Train Avg loss 1.576439 

Test Accuracy: 56.3%, Test Avg loss: 1.296274 

Epoch 5
-------------------------------
 Train Accuracy: 48.4%, Train Avg loss 1.536289 

Test Accuracy: 56.0%, Test Avg loss: 1.269391 

Epoch 6
-------------------------------
 Train Accuracy: 50.2%, Train Avg loss 1.497964 

Test Accuracy: 61.6%, Test Avg loss: 1.175473 

Epoch 7
-------------------------------
 Train Accuracy: 51.1%, Train Avg loss 1.475591 

Test Accuracy: 62.4%, Test Avg loss: 1.161380 

Epoch 8
--------------------------